The differences between the two groups of users can be seen by scrolling all the way to the bottom. The greatest difference that I can find is that the adopted users were much more likely to have joined via an invite (37% vs 57%) and much more likely to send an invite to another user (23% vs 12%).

In [184]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [185]:
user_engagement=pd.read_csv(r'takehome_user_engagement.csv')
users=pd.read_csv(r'takehome_users.csv',encoding='ISO-8859-1')

In [190]:
user_engagement['time_stamp']=pd.to_datetime(user_engagement.time_stamp)

In [191]:
import datetime
adopted_user=[]

total=0
for i in range(len(user_engagement)-3):
    if ((user_engagement.user_id.iloc[i]==user_engagement.user_id.iloc[i+3]) and (user_engagement.time_stamp.iloc[i+2]-user_engagement.time_stamp.iloc[i]).days<=7):
        adopted_user.append(1)
    else:
        adopted_user.append(0)
        
while (len(adopted_user)<len(user_engagement)):
    adopted_user.append(0)

In [192]:
user_engagement['adopted_user']=adopted_user

In [193]:
user_engagement=user_engagement.join(user_engagement[['user_id','adopted_user']].groupby('user_id').sum(),on='user_id',lsuffix='user_engagement')

In [194]:
user_engagement=pd.merge(user_engagement,users,left_on='user_id',right_on='object_id')

In [195]:
user_engagement=user_engagement.drop('adopted_useruser_engagement',axis=1)

In [196]:
user_engagement=user_engagement.drop('email',axis=1)

In [197]:
user_engagement=pd.merge(user_engagement,user_engagement['invited_by_user_id'].drop_duplicates(),left_on='user_id',right_on='invited_by_user_id',how='left')

In [198]:
user_engagement=user_engagement.rename({'invited_by_user_id_y':'sent_invite'},axis=1)

In [199]:
user_engagement['sent_invite']=user_engagement['sent_invite']>0

In [200]:
user_engagement['sent_invite']=user_engagement['sent_invite'].apply(lambda x: 1 if x else 0)

In [201]:
user_engagement=user_engagement.rename({'invited_by_user_id_x':'got_invite'},axis=1)

In [202]:
user_engagement['got_invite']=user_engagement['got_invite']>0
user_engagement['got_invite']=user_engagement['got_invite'].apply(lambda x: 1 if x else 0)

In [203]:
user_engagement=user_engagement.drop(['name','time_stamp','creation_time'],axis=1)

In [206]:
user_engagement=user_engagement.drop_duplicates()

In [210]:
user_engagement=pd.get_dummies(user_engagement)

In [211]:
full_table=pd.merge(users,user_engagement,left_on='object_id',right_on='user_id',how='left')

In [217]:
full_table=full_table.rename({'object_id_x':'user_id'},axis=1)

In [220]:
full_table=full_table.drop(['creation_time','name','email','creation_source','last_session_creation_time_x','opted_in_to_mailing_list_x','enabled_for_marketing_drip_x'],axis=1)

In [225]:
full_table=full_table.rename({'org_id_x':'org_id'},axis=1)

In [228]:
full_table=full_table.drop(['object_id_y'],axis=1)

In [230]:
full_table=full_table.rename({'last_session_creation_time_y':'last_session_creation_time','opted_in_to_mailing_list_y':'opted_in_to_mailing_list','enabled_for_marketing_drip_y':'enabled_for_marketing_drip'},axis=1)

In [233]:
full_table=full_table.rename({'org_id_y':'org_id'},axis=1)

In [235]:
full_table=full_table.drop(['invited_by_user_id'],axis=1)

In [237]:
full_table['adopted_user']=full_table['adopted_user'].apply(lambda x: 1 if x>0 else 0)

C:\Users\JackW\AppData\Local\Temp/ipykernel_24172/3709502818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['adopted_user']=full_table['adopted_user'].apply(lambda x: 1 if x>0 else 0)


In [246]:
full_table[full_table['adopted_user']==0].isna().sum()

user_id                                  0
org_id                                   0
user_id                               3177
visited                               3177
adopted_user                             0
last_session_creation_time            3177
opted_in_to_mailing_list              3177
enabled_for_marketing_drip            3177
org_id                                3177
got_invite                            3177
sent_invite                           3177
creation_source_GUEST_INVITE          3177
creation_source_ORG_INVITE            3177
creation_source_PERSONAL_PROJECTS     3177
creation_source_SIGNUP                3177
creation_source_SIGNUP_GOOGLE_AUTH    3177
dtype: int64

In [247]:
full_table[full_table['adopted_user']==1].isna().sum()

user_id                               0
org_id                                0
user_id                               0
visited                               0
adopted_user                          0
last_session_creation_time            0
opted_in_to_mailing_list              0
enabled_for_marketing_drip            0
org_id                                0
got_invite                            0
sent_invite                           0
creation_source_GUEST_INVITE          0
creation_source_ORG_INVITE            0
creation_source_PERSONAL_PROJECTS     0
creation_source_SIGNUP                0
creation_source_SIGNUP_GOOGLE_AUTH    0
dtype: int64

In [257]:
full_table = full_table.loc[:,~full_table.columns.duplicated()].copy()

In [261]:
full_table=full_table.fillna(0)

In [263]:
full_table[full_table['adopted_user']==0].describe()

,user_id,org_id,visited,adopted_user,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,got_invite,sent_invite,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
count,10394.000000,10394.000000,10394.000000,10394.0,1.039400e+04,10394.000000,10394.000000,10394.000000,10394.000000,10394.000000,10394.000000,10394.000000,10394.000000,10394.000000
mean,5993.826438,138.675582,0.694343,0.0,9.551555e+08,0.174043,0.104964,0.371272,0.121416,0.118434,0.252838,0.057822,0.154224,0.111026
std,3462.305029,123.910621,0.460707,0.0,6.339556e+08,0.379164,0.306522,0.483168,0.326626,0.323137,0.434660,0.233417,0.361180,0.314179
min,1.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2987.250000,26.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5997.500000,103.000000,1.000000,0.0,1.362196e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8987.750000,234.000000,1.000000,0.0,1.386654e+09,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,12000.000000,416.000000,1.000000,0.0,1.402067e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [264]:
full_table[full_table['adopted_user']==1].describe()

,user_id,org_id,visited,adopted_user,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,got_invite,sent_invite,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
count,1606.000000,1606.000000,1606.0,1606.0,1.606000e+03,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000,1606.000000
mean,6043.691158,162.653176,1.0,1.0,1.395700e+09,0.259651,0.155666,0.570984,0.231631,0.222291,0.348692,0.101494,0.183686,0.143836
std,3477.553247,123.028300,0.0,0.0,1.288262e+07,0.438580,0.362652,0.495090,0.422006,0.415915,0.476705,0.302076,0.387349,0.351032
min,2.000000,0.000000,1.0,1.0,1.341244e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3068.000000,53.000000,1.0,1.0,1.400070e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6047.000000,140.500000,1.0,1.0,1.401312e+09,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9106.250000,264.750000,1.0,1.0,1.401810e+09,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,11988.000000,415.000000,1.0,1.0,1.401926e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
